In [5]:
# -------
# IMPORT LIBRAIRIES
# -------
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression, Lasso
from python_module.pricing_model import SABRModel, BlackScholesModel

In [6]:
def compute_vol_bump(spot_change, implied_vol, spot_vol_correl, vol_of_vol):

    vol_change = (vol_of_vol / implied_vol) * spot_vol_correl * spot_change
    vol_bump = implied_vol * vol_change
    
    return vol_bump

In [7]:
def compute_slide_pnl(market_data_df, slide_list, slide_type='spot-vol'):
    results = list()

    T = market_data_df['T'].unique()[0]
    est_alpha, est_rho, est_nu = SABRModel.solve_parameters(F=F, T=T, strikes=market_data_df['K'].to_list(), market_vols=market_data_df['IV'].to_list())

    for slide in slide_list:
        
        for index in market_data_df.index:
            S0, K, T, r, IV, option_type, delta = market_data_df.loc[index, ['S0', 'K', 'T', 'r', 'IV', 'option_type', 'delta']]
            S0_bumped = S0 * (1+slide)
            delta_hedge_pnl = S0*delta*slide*-1
            
            if slide_type == 'spot-only':
                results.append({'slide': f'slide spot-only {slide:.2f}', 'index': index, 'delta_hedge_pnl': delta_hedge_pnl, 'price': BlackScholesModel.compute_option(S0_bumped, K, T, r, IV, option_type, False)})
            
            elif slide_type == 'spot-vol':
                vol_bump = compute_vol_bump(slide, IV, est_rho, est_nu)
                IV_bumped = IV + vol_bump
                results.append({'slide': f'slide spot-vol {slide:.2f}', 'index': index, 'delta_hedge_pnl': delta_hedge_pnl, 'price': BlackScholesModel.compute_option(S0_bumped, K, T, r, IV_bumped, option_type, False)})
            else:
                raise ValueError
    slide_pnl = pd.DataFrame(results).pivot_table(index='index', columns='slide', values='price').add(-market_data_df['price'], axis=0)
    delta_hedge_pnl = pd.DataFrame(results).pivot_table(index='index', columns='slide', values='delta_hedge_pnl')
    total_pnl = slide_pnl + delta_hedge_pnl

    return total_pnl.loc[market_data_df.index]

In [8]:
# -------
# GENERATE MARKET DATA AND GREEKS
# -------

# Explicit pricing parameters
S0 = F = 100
option_type = 'put'
time_to_maturity = 365

# Hidden pricing parameters
alpha = +0.1
beta  = +1.0
rho   = -0.5
nu    = +1

# Pre-processing
T = time_to_maturity / 250

# Pricing factory
market_data_list = list()
for K in np.linspace(start=80, stop=100, num=20, dtype=int):

    option_type = 'call' if K > 100 else 'put'

    IV = SABRModel.compute_sigma(F, K, T, alpha, beta, rho, nu)
    
    pricing_results = BlackScholesModel.compute_option(S0, K, T, 0, IV, option_type, True)
    
    market_data_list.append({

        # Descriptive option features
        'symbol': f"{time_to_maturity}_{K}_{option_type}",
        'option_type': option_type, 
        'time_to_maturity': time_to_maturity, 

        # others features
        'S0': S0,
        'r': 0,

        # Pricing option features
        'F': F,
        'K': K,
        'T': T, 
        'IV': IV,

        # Greeks and option price
        **pricing_results})

market_data_df = pd.DataFrame(market_data_list)
market_data_df = market_data_df.set_index('symbol')
market_data_df['weights'] = 0 # np.abs(np.random.normal(size=market_data_df.shape[0]))
market_data_df.loc[market_data_df.index[-5:], 'weights'] = 5

In [9]:
market_data_df

,option_type,time_to_maturity,S0,r,F,K,T,IV,price,delta,gamma,vega,theta,vanna,volga,weights
symbol,,,,,,,,,,,,,,,,
365_80_put,put,365,100,0,100,80,1.46,0.179260,1.517656,-0.127454,0.009634,0.252128,-0.006191,-1.073118,147.624983,0
365_81_put,put,365,100,0,100,81,1.46,0.175170,1.594872,-0.135362,0.010277,0.262827,-0.006307,-1.104831,147.033590,0
365_82_put,put,365,100,0,100,82,1.46,0.171107,1.677121,-0.143837,0.010964,0.273909,-0.006420,-1.134711,145.778098,0
365_83_put,put,365,100,0,100,83,1.46,0.167070,1.764877,-0.152930,0.011699,0.285374,-0.006531,-1.162126,143.783188,0
365_84_put,put,365,100,0,100,84,1.46,0.163059,1.858675,-0.162695,0.012485,0.297215,-0.006639,-1.186323,140.970643,0
365_85_put,put,365,100,0,100,85,1.46,0.159076,1.959116,-0.173192,0.013323,0.309422,-0.006743,-1.206403,137.261027,0
365_86_put,put,365,100,0,100,86,1.46,0.155121,2.066889,-0.184488,0.014217,0.321977,-0.006842,-1.221302,132.576192,0
365_87_put,put,365,100,0,100,87,1.46,0.151196,2.182775,-0.196655,0.015169,0.334854,-0.006935,-1.229762,126.842940,0
365_88_put,put,365,100,0,100,88,1.46,0.147303,2.307666,-0.209773,0.016182,0.348013,-0.007022,-1.230302,119.998193,0


In [10]:
est_alpha, est_rho, est_nu = SABRModel.solve_parameters(F=F, T=T, strikes=market_data_df['K'].to_list(), market_vols=market_data_df['IV'].to_list())
est_alpha, est_rho, est_nu

(0.0999999999752278, -0.49999999870951556, 1.0000000003950966)

In [11]:
# -------
# COMPUTE SLIDE
# -------
slide_df = compute_slide_pnl(market_data_df, np.linspace(start=-0.1, stop=0.1, num=10), slide_type='spot-only')

In [12]:
portfolio_slide = slide_df.multiply(market_data_df['weights'], axis=0).sum()
portfolio_slide.name = 'portfolio'

In [13]:
X = slide_df.transpose()
y = portfolio_slide
np.random.seed(42)
model = Lasso(alpha=0.008, fit_intercept=False, max_iter=1000000)
model.fit(X, y)
proxy_slide = slide_df.multiply(model.coef_, axis=0).sum()
proxy_slide.name = 'proxy'
model.coef_

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       5.56714614, 4.93595335, 4.36925573, 3.85561665, 6.21866609])

In [14]:
pd.concat([portfolio_slide, proxy_slide], axis=1)

,portfolio,proxy
slide,,
slide spot-only -0.01,0.432793,0.432636
slide spot-only -0.03,3.962327,3.959365
slide spot-only -0.06,11.147873,11.135112
slide spot-only -0.08,22.029632,21.995501
slide spot-only -0.10,36.542999,36.471623
slide spot-only 0.01,0.423742,0.423742
slide spot-only 0.03,3.720227,3.721481
slide spot-only 0.06,10.047691,10.054200
slide spot-only 0.08,19.093105,19.110848
